In [0]:
%run ./schema_definition

In [0]:
%run ./aws/s3_client

In [0]:
import json
from botocore.exceptions import NoCredentialsError
from datetime import datetime, timezone
from pyspark.sql.types import StructType, StructField, StringType, TimestampType
import boto3

from spark.session import get_spark_session

# Initialize Spark session
spark = get_spark_session()

# AWS client
s3 = get_s3_client()

# Step 2 - Load the config and store the values into different variables
config = load_config_txt("config.txt")
bucket = config["bucket"]
prefix = config["prefix"]
files_tracking = config["files_tracking"]
last_ingested_times = config["last_ingested_times"]

# Step 3 - create tracking tables for files which have been ingested and
# the timestamp of the last ingested files (if not exists)
create_table(files_tracking, files_tracking_schema)
create_table(last_ingested_times, last_ingested_times_schema)

# Step 4: Read the last ingested timestamp
last_ingested_df = spark.read.table(last_ingested_times)
# Extract the only timestamp value
last_ingested_times = last_ingested_df.collect()[0]["last_ingested_times"]
# Convert to UTC 
if last_ingested_times.tzinfo is None:
    last_ingested_times = last_ingested_times.replace(tzinfo=timezone.utc)


# Step 5: List all files in S3
get_s3_data(s3, bucket, prefix, files_tracking, main_schema,last_ingested_times, files_tracking_schema)



